In [1]:
import sqlite3
import pandas as pd
import csv

In [3]:
# Connect to SQLite database (create if not exists)
conn = sqlite3.connect('library11.db')
cursor = conn.cursor()

In [ ]:

# Drop existing tables if they exist
cursor.execute("DROP TABLE IF EXISTS BookInventory")
cursor.execute("DROP TABLE IF EXISTS Transactions")
cursor.execute("DROP TABLE IF EXISTS BookDetails")
cursor.execute("DROP TABLE IF EXISTS Books")


In [ ]:
# Create Books table with merged fields for category, department, and subject
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Books (
        BK_ID TEXT PRIMARY KEY NOT NULL,
        BK_NAME TEXT,
        AUTHOR_NAME TEXT,
        Edition TEXT,
        Date_Year TEXT,
        Publisher TEXT,
        Place TEXT,
        ISBN TEXT,
        Price TEXT,
        Pages TEXT,
        Category TEXT,
        Department TEXT,
        Subject TEXT
    )
''')

In [ ]:
# Create BookInventory table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS BookInventory (
        BK_ID TEXT PRIMARY KEY NOT NULL,
        Number_of_Copies INTEGER,
        FOREIGN KEY (BK_ID) REFERENCES Books(BK_ID)
    )
''')


In [ ]:

# Create BookInventory table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS BookInventory (
        BK_ID TEXT PRIMARY KEY NOT NULL,
        Number_of_Copies INTEGER,
        FOREIGN KEY (BK_ID) REFERENCES Books(BK_ID)
    )
''')

In [ ]:
# Create BookDetails table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS BookDetails (
        Asset_Code TEXT,
        Call_No TEXT,
        BK_ID TEXT NOT NULL,
        BK_STATUS TEXT,
        FOREIGN KEY (BK_ID) REFERENCES Books(BK_ID)
    )
''')

In [ ]:
# Create Transactions table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Transactions (
        Transaction_ID INTEGER PRIMARY KEY AUTOINCREMENT,
        BK_ID TEXT,
        CARD_ID TEXT,
        Bill_Date TEXT,
        Vendor TEXT,
        FOREIGN KEY (BK_ID) REFERENCES Books(BK_ID)
    )
''')


In [ ]:
# Create VIEW Library
cursor.execute('''
    CREATE VIEW IF NOT EXISTS Library AS
    SELECT
        Books.BK_NAME AS book_name,
        Books.BK_ID AS book_id,
        Books.AUTHOR_NAME AS author_name,
        BookDetails.BK_STATUS AS book_status,
        Transactions.CARD_ID AS card_id,
        Books.Category AS category,
        BookDetails.Asset_Code AS asset_code,
        Books.Department AS department,
        Books.Subject AS subject,
        Books.Edition AS edition,
        Books.Date_Year AS date_year,
        BookDetails.Call_No AS call_no, 
        Books.Publisher AS publisher,
        Books.Place AS place,
        Books.ISBN AS isbn,
        Books.Price AS price,
        Transactions.Bill_Date AS bill_date,
        Transactions.Vendor AS vendor,
        Books.Pages AS pages,
        BookInventory.Number_of_Copies AS num_copies
    FROM
        Books
    LEFT JOIN BookDetails ON Books.BK_ID = BookDetails.BK_ID
    LEFT JOIN Transactions ON Books.BK_ID = Transactions.BK_ID
    LEFT JOIN BookInventory ON Books.BK_ID = BookInventory.BK_ID
''')


In [ ]:
conn.commit()

In [ ]:
# Function to insert data into Books table
def insert_books_data(row):
    cursor.execute('''
            INSERT INTO Books (BK_ID, BK_NAME, AUTHOR_NAME, Edition, Date_Year, Publisher, Place, ISBN, Price, Pages, Category, Department, Subject) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['Access_No'], row['Title'], row['Authors'], row['Edition'], row['Date_Year'],
          row['Publisher'], row['Place'], row['ISBN'], row['Price'], row['Pages'],row['Category'],row['Department'],row['Subject']))


In [ ]:

# Function to insert data into BookDetails table
def insert_book_details_data(row):
    cursor.execute('''
        INSERT INTO BookDetails (Asset_Code, Call_No, BK_ID)
        VALUES (?, ?, ?)
    ''', (row['Asset_Code'], row['Call_No'], row['Access_No']))

In [ ]:
# Function to insert data into BookInventory table
def insert_book_inventory_data(row):
    cursor.execute('''
        INSERT INTO BookInventory (BK_ID, Number_of_Copies)
        VALUES (?, ?)
    ''', (row['Access_No'], row['copies']))

In [ ]:
# Function to insert data into Transactions table
def insert_transactions_data(row):
    cursor.execute('''
        INSERT INTO Transactions (BK_ID, Bill_Date, Vendor)
        VALUES (?, ?, ?)
    ''', (row['Access_No'], row['Bill_Date'], row['Vendor']))

In [ ]:
#Function to processing the data and call insertion functions
def process_csv_data():
    with open('C:/Users/Hunny/Downloads/updated_file.csv', newline='', encoding='latin-1') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Insert data into respective tables
            insert_books_data(row)
            insert_book_details_data(row)
            insert_book_inventory_data(row)
            insert_transactions_data(row)
            # Add other table insertions if necessary

    conn.commit()
    print("Data inserted successfully.")

In [ ]:
#call function to process data and insert into tables
process_csv_data()

In [4]:
#updating number of copies to one by default
cursor.execute("UPDATE BookInventory SET Number_of_Copies = 1 WHERE Number_of_Copies IS NULL OR Number_of_Copies NOT BETWEEN 2 AND 100;")

In [12]:
#trigger to prevent negative copies of books
cursor.execute("""CREATE TRIGGER prevent_negative_copies
BEFORE UPDATE ON BookInventory
FOR EACH ROW
WHEN NEW.Number_of_Copies < 0
BEGIN
    SELECT RAISE(ABORT, 'Cannot set Number_of_Copies below 0');
END;
""")

In [4]:
#for updating  number of copies on issueing
cursor.execute(""" 
            CREATE TRIGGER update_copies_on_issue
AFTER UPDATE ON Transactions
FOR EACH ROW
WHEN NEW.CARD_ID <> 'N/A' AND OLD.CARD_ID = 'N/A'
BEGIN
    UPDATE BookInventory
    SET Number_of_Copies = Number_of_Copies - 1
    WHERE BK_ID = NEW.BK_ID;
END;
""")

In [5]:
#for updating number of copies on returning
cursor.execute(""" 
CREATE TRIGGER update_copies_on_return
AFTER UPDATE ON Transactions
FOR EACH ROW
WHEN NEW.CARD_ID = 'N/A' AND OLD.CARD_ID <> 'N/A'
BEGIN
    UPDATE BookInventory
    SET Number_of_Copies = Number_of_Copies + 1
    WHERE BK_ID = NEW.BK_ID;
END;            
""")

In [5]:
conn.commit()

In [4]:

#Setting book status available by default
cursor.execute("UPDATE BookDetails SET BK_STATUS= 'Available' ")

In [4]:
#setting number of copies 1 by default
cursor.execute("UPDATE BookInventory SET Number_of_Copies = 1 ")

In [ ]:
#setting card id N/A by default
cursor.execute("UPDATE Transcations SET CARD_ID = 'N/A' ")

In [ ]:
conn.commit()

In [4]:
pd.read_sql("SELECT * FROM Library;",conn).head()

,book_name,book_id,author_name,book_status,card_id,category,asset_code,department,subject,edition,date_year,call_no,publisher,place,isbn,price,bill_date,vendor,pages,num_copies
0,Principles of economics,1,"Gregory, Mankiw N.",Available,N/A,Books,RVUN21LB0001,School of Economics,Economics,7th Ed,2015,330 GRE,Cengage learning india pvt. ltd,New Delhi,978-9386668035,899.0,10/25/2021 0:00,Book Paradise,"xxxii, 847",8
1,Microeconomics in context,9,"Goodwin, Neva R.",None,None,Books,RVUN21LB0009,School of Economics,Economics,2nd Ed,2008,339 GOO,PHI Learning Private Limited,New Delhi,9788120337473,350.0,10/29/2021 0:00,Book Paradise,"xxxiii,522",2
2,Macroeconomics in context,11,"Goodwin, Neva R.",Issued,None,Books,RVUN21LB0011,School of Economics,Economics,2nd Ed,2014,339 GOO,Routledge,New York,9781315702735,3295.0,10/29/2021 0:00,Book Paradise,"xxiv, 688",2
3,Economic development,13,"Michael, Todaro.",None,None,Books,RVUN21LB0013,School of Economics,Economics,12th Ed,2017,338.90091724 TOD,Pearson Education,Noida,978-9332585539,749.0,10/29/2021 0:00,Book Paradise,"xxvii,860",8
4,International economics,21,"Appleyard, Dennis R.",None,None,Books,RVUN21LB0021,School of Economics,Economics,8th Ed,2013,337 APP,Mcgrawhill,Chennai,9781259097423,925.0,10/29/2021 0:00,Book Paradise,"xxiv,824",2


In [6]:
conn.close()